In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import time
from sklearn import preprocessing

## Cost/activation

In [2]:
def cost_MSE(t,y_hat, derivative=0):
    if derivative:
            return -(t - y_hat)
    return np.mean(1/2*np.sum(np.power(t - y_hat, 2),\
                      axis=0))

def logistic_sigmoid(x, derivative=0):    
    sigm = 1/(1 + np.exp(-x))
    if len(sigm.shape) < 2:
        sigm = sigm.reshape(sigm.shape[0],1)
        
    if derivative:
        return sigm*(1. - sigm)
    return sigm

## NN backend

In [3]:
# >>>>>>>>>>>>>>>>>>> init_weights_biases >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# W, B = init_weights_biases(4, 3, [2,2])
#no_hidden_units: needs a list with at least one element
def init_weights_biases(no_of_features, no_outputs, no_hidden_units, seed=1):
    
    W = []
    B = []
    rows, columns = 0, 0 
    last = len(no_hidden_units)
    np.random.seed(seed)
    
    if no_hidden_units: #list is not empty
        for i in range(last+1):
            if i == 0: #first weight
                rows = no_hidden_units[i]
                columns = no_of_features
            elif i > 0 and i < last:
                rows = no_hidden_units[i]
                columns = no_hidden_units[i-1]
            else: #last
                columns = rows # list ran out of indeces, so use last one
                rows = no_outputs            

            W.insert(i, np.random.randn(rows, columns))
            B.insert(i, np.zeros((rows, 1)))
    else: # no hidden units (perceptron)
        W.insert(0, np.random.randn(no_outputs, no_of_features))
        B.insert(0, np.zeros((no_outputs, 1)))
    
    dummy_param = 0
    param = 0
    for i in range(len(W)):
        dummy_param = W[i].shape[0] * W[i].shape[1]
        param += dummy_param
        
#     W.append(param) #number of learnable weights
    
    return W, B, param

# >>>>>>>>>>>>>>>>>>> forward_prop >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    # W1, b1, W2, b2 = init_weights_biases(no_hidden_units=8)
    # Z, A, Y = forward_prop(W, B, X)
    # X has n features x M samples
def forward_prop(W, B, X):
    no_of_samples = X.shape[1]
     #last weight matrix, rows correspond to outputs
    no_of_outputs = W[-2].shape[0] #index -1 is the number of learnable weights
    
    Z = []
    A = []
    A.append(X) #first layer is an activation
    
    for i in range(len(W)): #to avoid the last two indeces
        Z.insert(i, W[i] @ A[i] + B[i])
        A.insert(i+1, logistic_sigmoid(Z[i]))
    
    Y = np.zeros((no_of_samples, no_of_outputs))
    #scaling to making the pair a probability
    Y = np.divide(A[i+1], np.sum(A[i+1], axis=0)) #comuns are the samples now
    return Z, A, Y

# >>>>>>>>>>>>>>>>>>> backprop >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# W1, b1, W2, b2 = init_weights_biases(no_hidden_units=8)
# A1, A2, Y = forward_prop(W1, b1, W2, b2, X)
# grad_mid_layer, grad_output = backprop(W2, A1, A2, X, Y, t)
# backprop(W2, A1, A2, X, Y, t)
def backprop(W, Z, A, Y_hat, T):
    
    output_index = len(W)-1 # if 3, starts at 2
    error = {}
    
    error_output = cost_MSE(T,Y_hat, derivative=1) * logistic_sigmoid(Z[-1], derivative=1)
    error[output_index] = error_output
    
    dJ_dW = {}
    for i in range(output_index-1,-1,-1):
         # doesn't get to W[0], so updated after the foor loop again
#         dJ_dW.insert(i+1, error[i+1] @ A[i+1].T)
        dJ_dW[i+1] = error[i+1] @ A[i+1].T
        
        error_dummy = (W[i+1].T @ error[i+1]) * logistic_sigmoid(Z[i], derivative=1)
#         error.insert(i, error_dummy)
        error[i] = error_dummy
    
    dJ_dW[0] = error[0] @ A[0].T
    
    return dJ_dW

## NN frontend 1 

    train, predict functions

In [ ]:
# >>>>>>>>>>>>>>>>>>> train >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    # all samples (X), 4 x 2, are fed
    # X: dataset, n samples x N features
    # T: binary labels, n labels x L number of ouputs
    # hidden_layers : list with number of neurons for each inner layer.
        # e.g. [3, 4] will yield two layers with 3 and 4 units respectively
    # this function needs n samples > 1 (batch optimization).
def train(X, T, hidden_layers=[2], epochs=500,\
          rho=.1, normalize_data=False, show_cost=0, seed=1, w_transfer=[]):    
    
    if normalize_data:
        scaler = preprocessing.StandardScaler()
        X = scaler.fit_transform(X)
    
    no_of_features = X.shape[1]
    no_samples = X.shape[0]
    no_outputs = T.shape[1]
    
    W, B, param = init_weights_biases(no_of_features,\
                                      no_outputs, hidden_layers, seed=1)
    
    if w_transfer:
        W=w_transfer
        
    Y_hat = np.zeros((no_outputs, X.shape[0]))    

    N = int(np.round(epochs/20))
    i_for_show_cost = np.round(np.linspace(0,epochs,N))
    
    ###NESTED function
    def display_NN_info():
        print("* NN ************************************")
        print("   no. inputs (layer 1): " + str(no_of_features) )
        for k in range(len(hidden_layers)):
            print("   layer " + str(k+2) + ": " + str(hidden_layers[k]) + " units")
        print("   output layer ("+ str(k+3) + "): " + str(no_outputs) )
        print("   learnable weights: " + str(param) )
        print("   max epochs: " + "{:,}".format(epochs) )
        print("   learning rate(rho): " + str(rho) )
        
    display_NN_info()
    time.sleep(4)
    #loop variables, along with other variables and lists (cost, accuracy)
    # that are returned by the function
    cost_final = []
    accuracy = []
    iter_passed = []
    match = 0
    idx_done = 0
    converged = False
    k_cost = 0
    
    for i in range(epochs):
        Z, A, Y_hat = forward_prop(W, B, X.T)
        dJ_dW = backprop(W, Z, A, Y_hat, T.T)
        
        sign = 1
    
        #grad descent
        for a in range(len(W)):
            W[a] = W[a] - sign*rho*dJ_dW[a]
        
        if match <=5:
            idx_done = i + 1
        
        if is_equal_in_boolean_terms(Y_hat.T, T):
            match += 1            
                
            if match > 6 and match < 7:
                idx_done = i + 1 # already predicts corretly all the time
            
        if match > 20: #seeing the match 100 times 
            converged = True #makes the predictions more confident
            break
            
        cost_final.insert(i,cost_MSE(T, Y_hat.T))
        accuracy.insert(i,calc_accuracy(T, Y_hat.T))
        iter_passed.insert(i,i)
        if show_cost and i_for_show_cost[k_cost] == i:
            print("   " + str(i) + " iteration, accuracy: "+ str(accuracy[i])+ "%")
            print("   cost: " + str(cost_final[i]))
            k_cost += 1        
        
    if converged:
        print("   Converged in " + str(idx_done) + " iterations")
    else:
        print("   Did not converge.")
    print()
    if show_cost:
        plt.scatter(iter_passed, cost_final, s=1, color="red")
        plt.title("iterations X cost")
        plt.xlabel("iterations")
        plt.ylabel("cost")
        
    return [W, B, Y_hat.T, X, cost_final, epochs,\
            idx_done, converged, rho, normalize_data, accuracy]

# >>>>>>>>>>>>>>>>>>> predict >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    # X: dataset, n samples x N features
    #  train_pkg: list with [W, B, Y, X, cost_final, epochs, idx_done, converged, rho, normalize_data, accuracy]
def predict(X, T, train_pkg):
    if len(X.shape) < 2:
        X = X.reshape(1,X.shape[0]) #for one sample
    
    normalized = train_pkg[-1]
    if normalized: #if the data has been normalized
        scaler = preprocessing.StandardScaler()
        X = scaler.fit_transform(X)
    
    Z, A, Y_hat = forward_prop(train_pkg[0], train_pkg[1], X.T)
    
    del Z, A
    return Y_hat.T, calc_accuracy(T, Y_hat.T)

def is_equal_in_boolean_terms(Y_hat, T):
    return np.array_equal(np.round(Y_hat), T)

def calc_accuracy(T, Y):
    matches = np.argmax(Y, axis=1) == np.argmax(T, axis=1)
    return len(matches[matches == True])/len(matches)*100

### NN functions 2 (helpers)

In [ ]:
# >>>>>>>>>>>>>>>>>>> train_all_gates >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def train_all_gates(X, t, hidden_layers=[3], iterations=500, rho=.01, normalize_data=False, print_cost=0):
    train_pkg_all_gates = {} #init dictionary

    for i in t:
        # hidden_layers = list of number of units for each layer. Minimum 1 [1]
                #train_gates[i] : [W, B, Y, X, cost_final, epochs, idx_done, \
                            #  converged, rho, normalize_data, accuracy]
        train_pkg_all_gates[i] = train(X, t[i],\
                                       hidden_layers = hidden_layers, \
                               epochs=iterations, rho=rho, \
                                       normalize_data=normalize_data, \
                                       show_cost=print_cost, seed=1)

    return train_pkg_all_gates

# >>>>>>>>>>>>>>>>>>> match_logic_gate >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def match_logic_gate(X, T, train_pkg):
    #y_hat is returned transposed already from predict
    Y_hat, accuracy = predict(X, T, train_pkg)
    
#   train_pkg : [W, B, Y, X, cost_final, epochs, idx_done,\
#                converged, rho, normalize_data, accuracy]
    # indeces used, especially:
    #                                 7. converged
    #                                 6. idx_done, 
    #                                 5. epochs
    #                                 8. rho
    match_pkg = [train_pkg[7], train_pkg[6],\
                  train_pkg[5], train_pkg[8],\
                  is_equal_in_boolean_terms(Y_hat, T), Y_hat]
    return match_pkg

# >>>>>>>>>>>>>>>>>>> match_all_gate_outputs >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def match_all_gate_outputs(X, T, train_pkg_all_gates):
    matches = {}

    for i in t:
        matches[i] = match_logic_gate(X, T[i], train_pkg_all_gates[i])

    return matches

def print_match(match):
    print(i + " converged: " + str(match[0]))
    print("===========================================")
    if match[1]:
        print("  iter. to converge: " + str(match[1]))
    else:
        print("  reached max. iter")
    print("  iter. max: " + str(match[2]))
    if match[4]:
        print("  ==== CORRECT prediction ==== ")
    else:
        print("  ==== INCORRECT prediction ==== ")
    print("  predicted y (y_hat): ")
    print(np.round(match[5]))
    print()

#### dataset / targets

X: possible inputs of a binary logic function.

t: dictionary with possible outputs for each logic gates. 
    4 binary ouputs to match NN's output probabilities of 0 or 1. 
    - if [p(0) p(1)] == [1 0] then probability of 0 == 1 && probability of 1 == 1

In [ ]:
X = np.array([[0,0],\
              [0,1],\
              [1,0],\
              [1,1]], dtype=np.float32)

t = { #dictionary for getting both the target logic values and the correlated string 
    # binary labels to represent the probabilities of 1 or 0 (first column is 0, 2nd 1)
    "AND": np.array([[1, 0],\
                     [1, 0],\
                     [1, 0],\
                     [0, 1]], dtype=np.float32),
    
    "NAND": np.array([[0, 1],\
                      [0, 1],\
                      [0, 1],\
                      [1, 0]], dtype=np.float32),
    
    "OR": np.array([[1, 0],\
                    [0, 1],\
                    [0, 1],\
                    [0, 1]], dtype=np.float32),
    
    "NOR": np.array([[0, 1],\
                     [1, 0],\
                     [1, 0],\
                     [1, 0]], dtype=np.float32),
    
#     "XOR": np.array([[1, 0],\
#                      [0, 1],\
#                      [0, 1],\
#                      [1, 0]], dtype=np.float32) }

    "XOR": np.array([[1, 0],\
                     [0, 1],\
                     [0, 1],\
                     [1, 0]], dtype=np.float32) }

## NN run

The Neural Network can be run with train() and then feeding the train_package to predict()

In [ ]:
train_pkg_all_gates = train_all_gates(X, t, hidden_layers=[2,4],\
                    iterations=10_000, normalize_data=True, rho=1, print_cost=1)

# (X, t, hidden_layers=[3], iterations=500, rho=.01, print_cost=0)
matches = match_all_gate_outputs(X, t, train_pkg_all_gates)

### matches
matches is a Python dictionary. For ex: 

    matches["OR"]
    returns a list:
            matches["OR"][0] == boolean (converged or not, True or False)
            matches["OR"][1] == idx_done, no. of iterations to converge
            matches["OR"][2] == total iterations (epochs)
            matches["OR"][3] == learning rate used(rho)
            matches["OR"][4] == matches with target (comparing both p(0) and p(1))
            matches["OR"][5] == predicted Y

In [ ]:
for i in matches:
    print_match(matches[i])
# predict(train_pkg_all_gates["XOR"])

# Extra credit - binary adder with carry

same network is used, just with a different dataset. The network adjusts inputs and outpus automatically. Just a new target is needed.

    Truth table for binary adder with Carry
    A B | Sum | C_out
    0 0 |   0 | 0
    0 1 |   1 | 0
    1 0 |   1 | 0
    1 1 |   0 | 1

In [7]:
T_ADD = {"ADDER": np.array([[0, 0],\
                [1, 0],\
                [1, 0],\
                [0, 1]], dtype=np.float32) }

## NN 2nd run (binary adder with sum and carry outputs

In [35]:
# W = train_pkg_all_gates["XOR"][0]
train_pkg_adder = train(X, T_ADD["ADDER"], hidden_layers=[3,6,3], epochs=10_000_000,\
                    rho=1, normalize_data=False, show_cost=1)#, w_transfer=W)   

match_adder = match_logic_gate(X, T_ADD["ADDER"], train_pkg_adder)
#     if match_adder[0]:
#         break

# match_adder = match_all_gate_outputs(X, T_ADD, train_pkg_adder)

* NN ************************************
   no. inputs (layer 1): 2
   layer 2: 3 units
   layer 3: 6 units
   layer 4: 3 units
   output layer (5): 2
   learnable weights: 48
   max epochs: 10,000,000
   learning rate(rho): 10
   0 iteration, accuracy: 25.0%
   cost: 0.548097556253651
   20 iteration, accuracy: 75.0%
   cost: 0.4695616253650968
   40 iteration, accuracy: 75.0%
   cost: 0.4695105713056389
   60 iteration, accuracy: 75.0%
   cost: 0.4694867881855483
   80 iteration, accuracy: 75.0%
   cost: 0.46947182813973687
   100 iteration, accuracy: 75.0%
   cost: 0.4694611055122915
   120 iteration, accuracy: 75.0%
   cost: 0.46945283253483805
   140 iteration, accuracy: 75.0%
   cost: 0.4694461401266391
   160 iteration, accuracy: 75.0%
   cost: 0.469440544852528
   180 iteration, accuracy: 75.0%
   cost: 0.46943575197488124
   200 iteration, accuracy: 75.0%
   cost: 0.4694315694098641
   220 iteration, accuracy: 75.0%
   cost: 0.46942786545446685
   240 iteration, accuracy: 75.

   3080 iteration, accuracy: 75.0%
   cost: 0.4693427970393646
   3100 iteration, accuracy: 75.0%
   cost: 0.4693426103430055
   3120 iteration, accuracy: 75.0%
   cost: 0.4693424248896034
   3140 iteration, accuracy: 75.0%
   cost: 0.46934224066267466
   3160 iteration, accuracy: 75.0%
   cost: 0.46934205764605996
   3180 iteration, accuracy: 75.0%
   cost: 0.469341875823917
   3200 iteration, accuracy: 75.0%
   cost: 0.4693416951807112
   3220 iteration, accuracy: 75.0%
   cost: 0.46934151570120686
   3240 iteration, accuracy: 75.0%
   cost: 0.4693413373704616
   3260 iteration, accuracy: 75.0%
   cost: 0.46934116017381766
   3280 iteration, accuracy: 75.0%
   cost: 0.4693409840968962
   3300 iteration, accuracy: 75.0%
   cost: 0.4693408091255868
   3320 iteration, accuracy: 75.0%
   cost: 0.46934063524604497
   3340 iteration, accuracy: 75.0%
   cost: 0.469340462444683
   3360 iteration, accuracy: 75.0%
   cost: 0.46934029070816474
   3380 iteration, accuracy: 75.0%
   cost: 0.46934

   6280 iteration, accuracy: 75.0%
   cost: 0.46932243515460154
   6300 iteration, accuracy: 75.0%
   cost: 0.4693223448360571
   6320 iteration, accuracy: 75.0%
   cost: 0.4693222548062553
   6340 iteration, accuracy: 75.0%
   cost: 0.4693221650633325
   6360 iteration, accuracy: 75.0%
   cost: 0.4693220756054448
   6380 iteration, accuracy: 75.0%
   cost: 0.4693219864307651
   6400 iteration, accuracy: 75.0%
   cost: 0.46932189753748405
   6420 iteration, accuracy: 75.0%
   cost: 0.4693218089238091
   6440 iteration, accuracy: 75.0%
   cost: 0.4693217205879662
   6460 iteration, accuracy: 75.0%
   cost: 0.46932163252819625
   6480 iteration, accuracy: 75.0%
   cost: 0.46932154474275745
   6500 iteration, accuracy: 75.0%
   cost: 0.4693214572299251
   6520 iteration, accuracy: 75.0%
   cost: 0.4693213699879908
   6540 iteration, accuracy: 75.0%
   cost: 0.4693212830152612
   6560 iteration, accuracy: 75.0%
   cost: 0.4693211963100594
   6580 iteration, accuracy: 75.0%
   cost: 0.46932

   9520 iteration, accuracy: 75.0%
   cost: 0.4693106281800247
   9540 iteration, accuracy: 75.0%
   cost: 0.4693105686182489
   9560 iteration, accuracy: 75.0%
   cost: 0.46931050918066586
   9580 iteration, accuracy: 75.0%
   cost: 0.4693104498667503
   9600 iteration, accuracy: 75.0%
   cost: 0.46931039067597985
   9620 iteration, accuracy: 75.0%
   cost: 0.46931033160783553
   9640 iteration, accuracy: 75.0%
   cost: 0.46931027266180164
   9660 iteration, accuracy: 75.0%
   cost: 0.46931021383736526
   9680 iteration, accuracy: 75.0%
   cost: 0.4693101551340177
   9700 iteration, accuracy: 75.0%
   cost: 0.4693100965512541
   9720 iteration, accuracy: 75.0%
   cost: 0.46931003808856986
   9740 iteration, accuracy: 75.0%
   cost: 0.46930997974546634
   9760 iteration, accuracy: 75.0%
   cost: 0.4693099215214463
   9780 iteration, accuracy: 75.0%
   cost: 0.4693098634160167
   9800 iteration, accuracy: 75.0%
   cost: 0.4693098054286873
   9820 iteration, accuracy: 75.0%
   cost: 0.46

   12700 iteration, accuracy: 75.0%
   cost: 0.4693024402405033
   12720 iteration, accuracy: 75.0%
   cost: 0.46930239547248664
   12740 iteration, accuracy: 75.0%
   cost: 0.4693023507739327
   12760 iteration, accuracy: 75.0%
   cost: 0.46930230614462315
   12780 iteration, accuracy: 75.0%
   cost: 0.46930226158433913
   12800 iteration, accuracy: 75.0%
   cost: 0.46930221709286246
   12820 iteration, accuracy: 75.0%
   cost: 0.4693021726699774
   12840 iteration, accuracy: 75.0%
   cost: 0.4693021283154684
   12860 iteration, accuracy: 75.0%
   cost: 0.46930208402912194
   12880 iteration, accuracy: 75.0%
   cost: 0.4693020398107231
   12900 iteration, accuracy: 75.0%
   cost: 0.4693019956600608
   12920 iteration, accuracy: 75.0%
   cost: 0.46930195157692284
   12940 iteration, accuracy: 75.0%
   cost: 0.4693019075611003
   12960 iteration, accuracy: 75.0%
   cost: 0.46930186361238124
   12980 iteration, accuracy: 75.0%
   cost: 0.46930181973055995
   13000 iteration, accuracy: 75

...

   129920 iteration, accuracy: 75.0%
   cost: 0.4692339437103693
   129940 iteration, accuracy: 75.0%
   cost: 0.4692339389390788
   129960 iteration, accuracy: 75.0%
   cost: 0.46923393416848286
   129980 iteration, accuracy: 75.0%
   cost: 0.46923392939857933
   130000 iteration, accuracy: 75.0%
   cost: 0.46923392462936864
   130020 iteration, accuracy: 75.0%
   cost: 0.46923391986085033
   130040 iteration, accuracy: 75.0%
   cost: 0.4692339150930238
   130060 iteration, accuracy: 75.0%
   cost: 0.4692339103258893
   130080 iteration, accuracy: 75.0%
   cost: 0.4692339055594473
   130100 iteration, accuracy: 75.0%
   cost: 0.4692339007936956
   130120 iteration, accuracy: 75.0%
   cost: 0.46923389602863663
   130140 iteration, accuracy: 75.0%
   cost: 0.4692338912642679
   130160 iteration, accuracy: 75.0%
   cost: 0.46923388650059045
   130180 iteration, accuracy: 75.0%
   cost: 0.46923388173760383
   130200 iteration, accuracy: 75.0%
   cost: 0.4692338769753083
   130220 iteratio

   133100 iteration, accuracy: 75.0%
   cost: 0.46923319364590743
   133120 iteration, accuracy: 75.0%
   cost: 0.4692331889822517
   133140 iteration, accuracy: 75.0%
   cost: 0.4692331843192571
   133160 iteration, accuracy: 75.0%
   cost: 0.4692331796569231
   133180 iteration, accuracy: 75.0%
   cost: 0.46923317499525036
   133200 iteration, accuracy: 75.0%
   cost: 0.4692331703342371
   133220 iteration, accuracy: 75.0%
   cost: 0.4692331656738848
   133240 iteration, accuracy: 75.0%
   cost: 0.46923316101419205
   133260 iteration, accuracy: 75.0%
   cost: 0.4692331563551595
   133280 iteration, accuracy: 75.0%
   cost: 0.4692331516967867
   133300 iteration, accuracy: 75.0%
   cost: 0.46923314703907276
   133320 iteration, accuracy: 75.0%
   cost: 0.46923314238201863
   133340 iteration, accuracy: 75.0%
   cost: 0.4692331377256237
   133360 iteration, accuracy: 75.0%
   cost: 0.46923313306988657
   133380 iteration, accuracy: 75.0%
   cost: 0.46923312841480896
   133400 iteratio

   136340 iteration, accuracy: 75.0%
   cost: 0.4692324466211368
   136360 iteration, accuracy: 75.0%
   cost: 0.4692324420620897
   136380 iteration, accuracy: 75.0%
   cost: 0.46923243750367266
   136400 iteration, accuracy: 75.0%
   cost: 0.4692324329458872
   136420 iteration, accuracy: 75.0%
   cost: 0.4692324283887309
   136440 iteration, accuracy: 75.0%
   cost: 0.46923242383220554
   136460 iteration, accuracy: 75.0%
   cost: 0.4692324192763095
   136480 iteration, accuracy: 75.0%
   cost: 0.4692324147210436
   136500 iteration, accuracy: 75.0%
   cost: 0.469232410166407
   136520 iteration, accuracy: 75.0%
   cost: 0.46923240561239943
   136540 iteration, accuracy: 75.0%
   cost: 0.46923240105902164
   136560 iteration, accuracy: 75.0%
   cost: 0.4692323965062731
   136580 iteration, accuracy: 75.0%
   cost: 0.469232391954153
   136600 iteration, accuracy: 75.0%
   cost: 0.46923238740266143
   136620 iteration, accuracy: 75.0%
   cost: 0.46923238285179814
   136640 iteration, 

   139440 iteration, accuracy: 75.0%
   cost: 0.46923174738697265
   139460 iteration, accuracy: 75.0%
   cost: 0.46923174292355946
   139480 iteration, accuracy: 75.0%
   cost: 0.4692317384607495
   139500 iteration, accuracy: 75.0%
   cost: 0.46923173399854295
   139520 iteration, accuracy: 75.0%
   cost: 0.4692317295369396
   139540 iteration, accuracy: 75.0%
   cost: 0.46923172507593913
   139560 iteration, accuracy: 75.0%
   cost: 0.4692317206155422
   139580 iteration, accuracy: 75.0%
   cost: 0.4692317161557466
   139600 iteration, accuracy: 75.0%
   cost: 0.4692317116965543
   139620 iteration, accuracy: 75.0%
   cost: 0.46923170723796415
   139640 iteration, accuracy: 75.0%
   cost: 0.46923170277997633
   139660 iteration, accuracy: 75.0%
   cost: 0.46923169832259015
   139680 iteration, accuracy: 75.0%
   cost: 0.46923169386580593
   139700 iteration, accuracy: 75.0%
   cost: 0.46923168940962356
   139720 iteration, accuracy: 75.0%
   cost: 0.4692316849540421
   139740 iterat

KeyboardInterrupt: 

In [ ]:
match_adder

In [ ]:
W = train_pkg_all_gates["XOR"][0]